# Initiation
Import needed packages and define some functions for preprocessing.

Make sure to have loaded all the needed modules before loading them here. This is done through e.g.:

```
module load py-h5py
module load py-tensorflow/1.8.0_py27
module load py-scipy
module load py-scikit-learn
module load py-scipystack
module load python
```

In [1]:
import sys
sys.path.append('../') #this allows us to import files form the ./stroke-deep-learning/src/-folder
from src import utils
from __future__ import division
from os import listdir
import utils
import h5py
import numpy as np
import pandas as pd
import os
import sys
from scipy.signal import butter, sosfilt, welch
from scipy.interpolate import interp1d

def determine_artefact_segments(x, fs, window_size, factors):   
    f, t, Sxx = signal.spectrogram(x, fs, nperseg=window_size, noverlap=0)
    ratio_high_frequency = np.sum(Sxx[f > 45,:],axis=0) / np.sum(Sxx[f < 20,:],axis=0)
    power_high_frequency = np.sum(Sxx[f > 45,:],axis=0)
    cond_ratio = ratio_high_frequency > np.min(ratio_high_frequency)*factors[0]
    cond_power = power_high_frequency > np.min(power_high_frequency)*factors[1]
    cond = cond_ratio+cond_power
    artefact_idx = cond != 0
    return t, artefact_idx

def remove_artefacts(x, fs):
    window_length=5
    window_size = int(5*fs)
    indices = []
    tt = np.arange(0, x.shape[1]//fs,1/fs)
    for i in range(x.shape[0]):
        t,idx = determine_artefact_segments(np.squeeze(x[i,:]), fs, window_size,factors=[1e4,1e4])
        f = interp1d(t, idx,'nearest', fill_value='extrapolate')
        indices.append(f(tt).tolist())
    i = np.sum(np.stack(indices),axis=0) == 0
    out = x[:,i]
    print('Before: {}, after: {}'.format(x.shape, out.shape))
    return out

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], btype='band', output='sos')
    return sos
def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    sos = butter_bandpass(lowcut, highcut, fs, order=order)
    y = sosfilt(sos, data)
    return y

# Determine path to resources that we can define relatively based on their placement in the git repo
notebook_folder = os.getcwd()
resources_folder = os.path.abspath(os.path.join(notebook_folder, '../resources'))

print('Currently assuming resources are stored at:')
print(resources_folder)

Currently assuming resources are stored at:
/home/users/rmth/stroke-deep-learning/resources


# Setup paths and similar constants
Setup all the paths and other settings. Some need to be hardcoded to the user running it, and so should be changed from their current setup. Thus block of code should specify:
* which folder the EDFs are stored in
* where the hypnograms (if relevant) are stored
* which channels to load and how to orderthem/what to call them
* how long of a duration the epoching for the output should be
* where to output the processed data
* which cohort we're creating files for
* if we're doing simlated data
* how we're doing the rescaling
Note that the channel alias file needs to be manually changed to fit the need for which files are loaded and then stored in the designated output folder as a json. The json for channel aliases for the SHHS-Sherlock mode when a user runs this is therefore stored at:
`/scratch/users/[user-name]/processed_shhs_data/signal_labels.json`
and would look like:
~~~~
{
    "categories": [
        "eeg1",
        "eeg2"
    ],
    "edfFiles": [
        "/home/rasmus/Desktop/shhs_subset/control/shhs1-203490.EDF",
        "/home/rasmus/Desktop/shhs_subset/control/shhs1-205226.EDF",
        "/home/rasmus/Desktop/shhs_subset/control/shhs1-205256.EDF",
        "/home/rasmus/Desktop/shhs_subset/control/shhs1-203101.EDF",
        "/home/rasmus/Desktop/shhs_subset/control/shhs1-203291.EDF",
        "/home/rasmus/Desktop/shhs_subset/control/shhs1-205122.EDF",
        "/home/rasmus/Desktop/shhs_subset/control/shhs1-205328.EDF",
        "/home/rasmus/Desktop/shhs_subset/control/shhs1-204884.EDF",
        "/home/rasmus/Desktop/shhs_subset/control/shhs1-203629.EDF",
        "/home/rasmus/Desktop/shhs_subset/control/shhs1-203716.EDF"
    ],
    "eeg1": [
        "EEG"
    ],
    "eeg2": [
        "EEG 2",
	"EEG sec",
	"EEG(SEC)",
	"EEG(sec)",
	"EEG2"
    ],
    "pathname": "/home/rasmus/Desktop/shhs_subset/control"
} 
~~~~ 

This edFiles part is not important, as it is just a note on where the EDFs were stored when the tool for making the aliases was run. The tool used is stored in the repo as:


`./stroke-deep-learning/src/channel_label_identifier.py`

and was created by [Hyatt Moore IV](https://github.com/informaton)

In [2]:
# todo: consider implementing rejection of noise epochs
debugging = False
multimodal = False
simulated_data = False
revised_preprocessing = False
rescale_mode = 'soft'
#cohort = 'SHHS-Sherlock'
cohort = 'SHHS-Sherlock-matched'
#cohort = 'Simulated-Sherlock'

if cohort == 'SSC':
    edf_folder = '/home/rasmus/Desktop/SSC/raw/edf/'
    hypnogram_folder = '/home/rasmus/Desktop/SSC/raw/hypnograms/'
    epoch_duration = 5
    channels_to_load = {'C3': 0, 'C4': 1, 'Fz': 2, 'ROC': 3, 'LOC': 4, 'Chin': 5}
    output_folder = '/home/rasmus/Desktop/SSC/processed_data/'
    IDs = listdir(edf_folder)
    channel_alias = utils.read_channel_alias(edf_folder+'signal_labels.json')
elif cohort == 'SHHS-Sherlock':
    epoch_duration = 5*60
    edf_folder = '/scratch/PI/mignot/nsrr/shhs/polysomnography/edfs/shhs1/'
    hypnogram_folder = None
    df = pd.read_csv('/home/users/rmth/stroke-deep-learning/IDs.csv')
    IDs = np.asarray(df['IDs'])
    group = np.asarray(df['group'])
    if multimodal:
        channels_to_load = {'eeg1': 0, 'eeg2': 1, 'ecg': 2, 'pulse': 3}
        output_folder = '/scratch/users/rmth/processed_shhs_data_multimodal/'
        channel_alias = utils.read_channel_alias(output_folder+'signal_labels_multimodal.json')
    else:
        channels_to_load = {'eeg1': 0, 'eeg2': 1}
        if revised_preprocessing:
            output_folder = '/scratch/users/rmth/processed_shhs_data_revised/'
        else:
            output_folder = '/scratch/users/rmth/processed_shhs_data/'
        channel_alias = utils.read_channel_alias(output_folder+'signal_labels.json')
elif cohort == 'SHHS-Sherlock-matched':
    epoch_duration = 5*60
    edf_folder = '/scratch/PI/mignot/nsrr/shhs/polysomnography/edfs/shhs1/'
    hypnogram_folder = None
    df = pd.read_csv(resources_folder + '/ids/matched_controls.csv')
    IDs = np.asarray(df['conIDs'])
    IDs = [int(e) for e in IDs]
    group = np.asarray(np.zeros(len(IDs)))
    if multimodal:
        channels_to_load = {'eeg1': 0, 'eeg2': 1, 'ecg': 2, 'pulse': 3}
        output_folder = '/scratch/users/rmth/processed_shhs_data_multimodal/'
        channel_alias = utils.read_channel_alias(output_folder+'signal_labels_multimodal.json')
    else:
        channels_to_load = {'eeg1': 0, 'eeg2': 1}
        if revised_preprocessing:
            output_folder = '/scratch/users/rmth/processed_shhs_data_revised/matched_controls/'
        else:
            output_folder = '/scratch/users/rmth/processed_shhs_data/matched_controls/'
        channel_alias = utils.read_channel_alias(output_folder+'signal_labels.json')
elif cohort == 'SHHS':
    epoch_duration = 5*60
    edf_folder = '/home/rasmus/Desktop/shhs_subset/'
    hypnogram_folder = None
    control = listdir(edf_folder + 'control')
    stroke = listdir(edf_folder + 'stroke')
    if debugging:
        control = control[0:1]
        stroke = stroke[0:1]
    group = np.concatenate((np.zeros(shape=len(control)),
                            np.ones(shape= len(stroke))))
    IDs = control+stroke
    if multimodal:
        channels_to_load = {'eeg1': 0, 'eeg2': 1, 'ecg': 2, 'pulse': 3}
        output_folder = '/home/rasmus/Desktop/shhs_subset/processed_data_multimodal/'
        channel_alias = utils.read_channel_alias(edf_folder+'signal_labels_multimodal.json')
    elif simulated_data:
        output_folder = '/home/rasmus/Desktop/shhs_subset/simulated_data/'
    else:
        channels_to_load = {'eeg1': 0, 'eeg2': 1}
        output_folder = '/home/rasmus/Desktop/shhs_subset/processed_data/'
        channel_alias = utils.read_channel_alias(edf_folder+'signal_labels.json')
elif cohort == 'Simulated-Sherlock':
    epoch_duration = 5*60
    edf_folder = '/scratch/PI/mignot/nsrr/shhs/polysomnography/edfs/shhs1/'
    hypnogram_folder = None
    df = pd.read_csv('/home/users/rmth/stroke-deep-learning/IDs.csv')
    IDs = np.asarray(df['IDs'])
    group = np.asarray(df['group'])
    channels_to_load = {'eeg1': 0, 'eeg2': 1}
    output_folder = '/scratch/users/rmth/simulated_data/'
    channel_alias = utils.read_channel_alias(output_folder+'signal_labels.json')


## Check that the following folders and outputs correspond to your expectations!

In [3]:
print('## Cohort:')
print(cohort)
print('## Data folder:')
print(edf_folder)
print('## Hypnogram_folder:')
print(hypnogram_folder)
print('## Output folder:')
print(output_folder)
print('## Channel alises:')
print(channel_alias)
print('## IDs')
print(IDs)

## Cohort:
SHHS-Sherlock-matched
## Data folder:
/scratch/PI/mignot/nsrr/shhs/polysomnography/edfs/shhs1/
## Hypnogram_folder:
None
## Output folder:
/scratch/users/rmth/processed_shhs_data/matched_controls/
## Channel alises:
{u'EEG(sec)': u'eeg2', u'EEG(SEC)': u'eeg2', u'EEG sec': u'eeg2', u'EEG 2': u'eeg2', u'EEG': u'eeg1', u'EEG2': u'eeg2'}
## IDs
[200021, 200316, 200379, 200388, 200426, 200436, 200787, 200803, 200950, 200999, 201010, 201057, 201063, 201092, 201284, 201327, 201400, 201401, 201509, 201536, 201632, 201640, 201659, 201666, 201781, 201808, 201835, 201837, 201887, 201912, 201990, 201992, 202111, 202119, 202152, 202161, 202218, 202229, 202348, 202377, 202378, 202421, 202431, 202432, 202536, 202537, 202576, 202630, 202640, 202664, 202714, 202785, 202829, 202839, 202849, 202930, 202937, 202988, 203102, 203152, 203196, 203350, 203891, 203988, 204149, 204214, 204223, 204502, 204520, 204566, 205121, 205186, 205199, 205352, 205403, 205715]


In [6]:
# If the processing crashes for some reason along with way,
# it can be nice to not redo the entire thing for all files.
# Here we check which we don't need to do.
# However, if you want to do this, you need to add a line in 
# in the clock below to "continue" if the ID is in done_recently.
import time
path = output_folder
done_recently = []
now = time.time()
for f in os.listdir(path):
    if os.stat(os.path.join(path,f)).st_mtime > now - 0.5 * 86400:
        done_recently.append(f)
print('Checking if certain files have already recently been run. File IDs recently processed:')
print(done_recently)

Checking if certain files have already recently been run. File IDs recently processed:
['shhs1-201968.hpf5']


# Processing
No we're actually doing the processing. Note that the filtering is done below with a bandpass filter with cutoffs at .3 and 40 Hz. The code for using hypnograms are also only fit for used with the SSC data (Stanford Sleep Clinic).

This implementation is a solution using try-catch, which is a poor way of handling errors in handling the EDFs, so make sure to check that what you output is in accordance with what you expect!

> Someone (or automatic cleanup) has removed the SHHS EDFs from the mignot scratch, so only 4 remain. I've tested with one of them below, but to use the code again, the files needs to be reuploaded. -rmth


In [5]:
debug = True # THIS SHOULD BE FALSE IF YOURE DOING SOMETHING FOR REAL

if simulated_data:
    flag = 0
    n_records = 0
    n_records_per_class = 20
for counter, ID in enumerate(IDs):
    if cohort == 'SHHS-Sherlock' or cohort == 'SHHS-Sherlock-matched' or cohort == 'Simulated-Sherlock':
            if 'shhs1-' + str(ID) + ".hpf5" in done_recently:
                print('Done recently, skipping.')
                continue
    
    if simulated_data:
        if flag == 0:
            if int(group[counter]) == 1:
                continue
            else:
                n_records += 1
                if n_records == n_records_per_class:
                    flag = 1
                    n_records = 0
        elif flag == 1:
            if int(group[counter]) == 0:
                continue
            else:
                n_records += 1
                if n_records == n_records_per_class:
                    flag = 2
                    n_records = 0
        elif flag == 2:
            break
    
    try:
        #print('***WARNING: THIS IS DEBUGGING, BECAUSE EDFS HAVE BEEN REMOVED FROM SHERLOCK AND ONLY THIS AND A FEW OTHERS REMAIN FOR RMTH TO TEST')
        #print(ID)
        if debug:
            ID = 201968
            debug = False
            
        if ID != 201968:
            continue
        print('Made it here')
        #if ID != 202345: # Different sampling frequency
        #    continue
        #if counter != 3:
        #    continue
            
        print('Processing: ' + str(ID) + ' (number ' + str(counter+1) + ' of ' + str(len(IDs)) + ').')
        if cohort == 'SSC':
            filename = edf_folder + ID
        elif cohort == 'SHHS-Sherlock' or cohort == 'SHHS-Sherlock-matched' or cohort == 'Simulated-Sherlock':
            filename = edf_folder + 'shhs1-' + str(int(ID)) + '.edf'
        elif cohort == 'SHHS':
            if group[counter] == 1:
                filename = edf_folder + 'stroke/' + ID
            else:
                filename = edf_folder + 'control/' + ID

                
        try:
            data = utils.load_edf_file(filename, channels_to_load,
                                       cohort = cohort,
                                       channel_alias = channel_alias)
        except Exception as e:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(exc_type, fname, exc_tb.tb_lineno)
            print(e)
            print('    EDF error (loading failed).')
            continue
        if data == -1:
            print('    Ignoring this subject due to different header setup.')
            continue

        if revised_preprocessing:
            z = remove_artefacts(data['x'],data['fs'])
            x = butter_bandpass_filter(z, .3, 40, data['fs'], order=32)
        else:
            x = butter_bandpass_filter(data['x'],0.3, 40.0, data['fs'], order=8)
            
        n = x.shape[1]
        epoch_samples = epoch_duration * data['fs']
        n_epochs = n // epoch_samples
        if hypnogram_folder:
            filename = hypnogram_folder + ID[:-4] + '.STA'
            try:
                hypnogram = utils.load_hypnogram_file(filename)
                if epoch_duration != 30:
                    hypnogram = np.repeat(hypnogram, repeats=30 // epoch_duration)
                if n != hypnogram.shape[0]:
                    hypnogram = hypnogram[:n_epochs]
            except:
                print('    Hypnogram error')
                continue

        epoched = np.zeros((int(len(channels_to_load)), int(n_epochs), int(epoch_samples)))
        for i in range(len(channels_to_load)):
            epoched[int(i), :, :] = np.asarray(list(zip(*[iter(x[int(i),:])] * int(epoch_samples))))
            
        if simulated_data:
            noise = np.random.normal(0, .25, epoched.shape)
            noise_complex = utils.add_known_complex(noise, data['fs'], group = int(group[counter]))
            x = noise_complex 
        else:
            x = utils.rescale(epoched, data['fs'], rescale_mode)

        if cohort == 'SSC':
            stages_to_use = [5,2]
            stage_names = ['wake','n1','n2','n3','n4','rem','unknown','artefact']
            for group, stage in enumerate(stages_to_use):
                output_file_name = output_folder +  ID[:-4] + '_' + stage_names[stage] + ".hpf5"
                with h5py.File(output_file_name, "w") as f:
                    dset = f.create_dataset("x", data=x[:, hypnogram == stage, :], chunks=True)
                    f['fs'] = data['fs']
                    f["group"] = group
        elif cohort == 'SHHS-Sherlock' or cohort == 'SHHS-Sherlock-matched' or cohort == 'Simulated-Sherlock':
            output_file_name = output_folder + 'shhs1-' + str(ID) + ".hpf5"
            with h5py.File(output_file_name, "w") as f:
                dset = f.create_dataset('x', data=x, chunks=True)
                f['fs'] = data['fs']
                f['group'] = int(group[counter])
        elif cohort == 'SHHS':
            output_file_name = output_folder +  ID[:-4] + ".hpf5"
            with h5py.File(output_file_name, "w") as f:
                dset = f.create_dataset("x", data=x, chunks=True)
                f['fs'] = data['fs']
                f["group"] = group[counter]
        print('Actually processed {}'.format(filename))
    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(exc_type, fname, exc_tb.tb_lineno)
        print(e)
        print('Error happened while processing: {}'.format(str(filename)))

print("All files processed (all processed will be mentioned, if IDs are listed, something went wrong).")

Made it here
Processing: 201968 (number 1 of 76).
Actually processed /scratch/PI/mignot/nsrr/shhs/polysomnography/edfs/shhs1/shhs1-201968.edf
All files processed (all processed will be mentioned, if IDs are listed, something went wrong).
